In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import joblib

Étape 2 : Charger les données

In [2]:
# Charger les données
data = pd.read_csv("spam.csv", encoding='latin-1')  # Utiliser 'latin-1' pour éviter les erreurs d'encodage

# Sélectionner les colonnes pertinentes
data = data[['v1', 'v2']]

# Renommer les colonnes pour plus de clarté
data.columns = ['label', 'text']

# Afficher les premières lignes pour vérifier
print(data.head())

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


Étape 3 : Nettoyer les données

In [15]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

# Télécharger les ressources nécessaires de NLTK
nltk.download('wordnet')
nltk.download('stopwords')

# Initialiser le lemmatiseur et les stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les caractères spéciaux
    text = re.sub(r'\W', ' ', text)
    # Supprimer les espaces multiples
    text = re.sub(r'\s+', ' ', text)
    # Lemmatisation et suppression des stopwords
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admis\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admis\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Étape 4 : Vectoriser les textes avec TF-IDF


In [16]:
# Initialiser le vectoriseur TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))

# Vectoriser les textes
X = vectorizer.fit_transform(data['text'])

# Convertir les étiquettes en 0 (ham) et 1 (spam)
y = data['label'].map({'ham': 0, 'spam': 1})

Étape 5 : Diviser les données en ensembles d'entraînement et de test

In [17]:
# Diviser les données (80% entraînement, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Étape 6 : Entraîner un modèle SVM


In [18]:
# Initialiser le modèle SVM
model = SVC(kernel='linear', random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

Étape 7 : Évaluer le modèle


In [19]:
# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_matrix(y_test, y_pred))

# Afficher le rapport de classification
print("\nRapport de classification :")
print(classification_report(y_test, y_pred))

Matrice de confusion :
[[961   4]
 [ 19 131]]

Rapport de classification :
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.97      0.87      0.92       150

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115



Étape 8 : Sauvegarder le modèle et le vectoriseur


In [20]:
# Sauvegarder le modèle
joblib.dump(model, "spam_classifier.pkl")

# Sauvegarder le vectoriseur
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

In [22]:
# Obtenir les noms des caractéristiques (mots)
feature_names = vectorizer.get_feature_names_out()

# Obtenir les coefficients du modèle SVM (pour la classe spam)
if hasattr(model, 'coef_'):
    coefficients = model.coef_[0]
else:
    coefficients = model.best_estimator_.coef_[0]

# Associer les mots à leurs coefficients
word_importance = list(zip(feature_names, coefficients))

# Trier par importance (coefficients les plus élevés pour les spams)
word_importance.sort(key=lambda x: x[1], reverse=True)

# Afficher les 10 mots les plus importants pour les spams
print("Top 10 mots importants pour les spams :")
for word, coef in word_importance[:10]:
    print(f"{word}: {coef}")

Top 10 mots importants pour les spams :
00:   (0, 1142)	0.22700826362146512
  (0, 4661)	0.220049539469254
  (0, 4660)	0.220049539469254
  (0, 4390)	0.220049539469254
  (0, 3994)	0.220049539469254
  (0, 3442)	0.22517082317983017
  (0, 3424)	0.220049539469254
  (0, 3423)	0.220049539469254
  (0, 2727)	0.220049539469254
  (0, 486)	0.2157195557884666
  (0, 46)	0.220049539469254
  (0, 45)	0.2157195557884666
  (0, 3824)	0.6744980020882821
  (0, 3505)	0.22483266736276072
  (0, 1621)	0.23268279861684263
  (0, 1449)	0.23268279861684263
  (0, 4281)	0.0803827555624305
  (0, 2817)	0.07855453070247904
  (0, 1908)	0.08262032195350477
  (0, 1791)	0.08262032195350477
  (0, 1496)	0.28146024555837246
  (0, 1018)	0.08262032195350477
  (0, 915)	0.07448873945145332
  (0, 96)	0.07855453070247904
  (0, 7)	0.08262032195350477
  :	:
  (0, 2889)	-0.002287007086134357
  (0, 2747)	-0.11243364118981489
  (0, 2496)	-0.36021155887569295
  (0, 2494)	-0.25339586629569966
  (0, 2456)	-0.09291026696962235
  (0, 1873)	-0.

Étape 9 : Tester le modèle avec de nouveaux exemples


In [21]:
# Charger le modèle et le vectoriseur
model = joblib.load("spam_classifier.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Définir une fonction pour prédire si un SMS est un spam
def predict_spam(text):
    # Nettoyer et vectoriser le texte
    text_cleaned = [clean_text(text)]
    text_vectorized = vectorizer.transform(text_cleaned)

    # Faire une prédiction
    prediction = model.predict(text_vectorized)
    return "Spam" if prediction[0] == 1 else "Non-Spam"

# Tester un exemple 
test_sms = "Congratulations Adam! You've won a $1000 Walmart gift card. Click here to claim your prize."
print(predict_spam(test_sms))  # Doit retourner "Spam"

# Tester un autre exemple
test_sms_2 = "Hey bro can i scam you?"
print(predict_spam(test_sms_2))  # Doit retourner "Non-Spam"

Spam
Non-Spam
